In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

examples = [
    {
        "question": "탑건",
        "answer": """
            🛩️👨‍✈️🔥
        """
    }, 
    {
        "question": "대부",
        "answer": """
            👨‍👨‍👦🔫🍝
        """
    }    
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{answer}"),
    ]
)
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화 제목을 듣고 세개의 이모티콘으로 표현해야해"),
        # ("system", "너는 대화를 도와줘야 해"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie}")
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"movie": question})
    memory.save_context(
        {"input":question},
        {"output": result.content}
    )
    print(result)

invoke_chain("신세계")
invoke_chain("앞에서 말한 영화제목은 뭐지?")

content='😱🔫🕵️\u200d♂️'
content='죄송해요, 방금 말씀해주신 영화 제목은 "신세계"에요.'
